In [128]:
from astropy.time import Time
import numpy as np
import pandas as pd
import astropy.units as u

## Load data

In [106]:
fink_grb_data_path = "/user/julien.peloton/fink_grb/"

In [107]:
df = spark.read\
.option("basePath", fink_grb_data_path)\
.format("parquet")\
.load([fink_grb_data_path + "gcn_storage/raw"]).toPandas()
df = df.sort_values("triggerTimejd")

## Simple stats

In [139]:
last_day = df["triggerTimeUTC"].values[-1]
print("first gcn date: {}\nlast gcn date: {}".format(
    df["triggerTimeUTC"].values[0], 
    last_day
))

first gcn date: 2023-06-15T18:08:26.002000000
last gcn date: 2023-07-03T10:48:38.888000000


#### GCN of the last day

In [142]:
df[df["triggerTimeUTC"] == last_day]

,observatory,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day,dayjd
74,LVK,H1_L1,gw,,S230703p,83.144531,32.442866,2.121365e+07,2023-07-03 12:53:38.737908,2.460129e+06,2023-07-03 10:48:38.888,"{""alert_type"": ""PRELIMINARY"", ""time_created"": ...",2023,7,3,2460129.0


### Nb gcn / day

In [109]:
df["dayjd"] = df["triggerTimejd"].round()
res_gb = df.groupby("dayjd").count()["observatory"]
"{:.3f} ± {:.3f} gcn/day".format(res_gb.mean(), res_gb.std())

'9.842 ± 3.701 gcn/day'

### Nb gcn / observatory

In [110]:
df.groupby("observatory").count()

,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day,dayjd
observatory,,,,,,,,,,,,,,,
Fermi,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35
ICECUBE,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
INTEGRAL,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
LVK,139,139,139,139,139,139,139,139,139,139,139,139,139,139,139
SWIFT,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


### Nb gcn / day / observatory

In [114]:
observatories = df["observatory"].unique()

In [138]:
for obs in observatories:
    nb_day = []
    for day in df["dayjd"]:
        nb_day.append(len(df[(df["dayjd"] == day) & (df["observatory"] == obs)]))
    
    print("{}: {:.3f} ± {:.3f} gcn/day".format(obs, np.mean(nb_day), np.std(nb_day)))
    print()

LVK: 8.299 ± 2.935 gcn/day

Fermi: 1.989 ± 1.473 gcn/day

INTEGRAL: 0.540 ± 0.711 gcn/day

SWIFT: 0.203 ± 0.402 gcn/day

ICECUBE: 0.128 ± 0.334 gcn/day



### Nb gcn / instruments

In [28]:
for obs in observatories:
    df_tmp = df[df["observatory"] == obs]
    print("--- {} ---".format(obs))
    gb_col = "instrument"
    if obs == "ICECUBE":
        gb_col = "event"
    print(df_tmp.groupby(gb_col).count()["triggerId"].to_markdown())
    print("\n")

--- LVK ---
| instrument          |   triggerId |
|:--------------------|------------:|
| H1                  |           1 |
| H1_L1               |         137 |
| MbtaL-Hon-clustered |           1 |


--- Fermi ---
| instrument   |   triggerId |
|:-------------|------------:|
| GBM          |          35 |


--- SWIFT ---
| instrument   |   triggerId |
|:-------------|------------:|
| BAT          |           1 |
| XRT          |           2 |


--- ICECUBE ---
| event   |   triggerId |
|:--------|------------:|
| Cascade |           2 |


--- INTEGRAL ---
| instrument   |   triggerId |
|:-------------|------------:|
| Weak         |           8 |




### GCN latency

Warning: this is the latency between the trigger time and the last gcn emitted for each events. This is not the latency computed with the first gcn emitted just after the emission.

In [64]:
df["jdAckTime"] = Time(df["ackTime"].values).jd
df["latency"] = (df["jdAckTime"] - df["triggerTimejd"]) * 24

#### GCN latency global

In [65]:
"{:.3f} ± {:.3f} hours".format(df["latency"].mean(), df["latency"].std())

'8.070 ± 11.644 hours'

In [66]:
df.groupby("observatory").agg(
    latency_mean=("latency", np.mean),
    latency_std=("latency", np.std),
    latency_min=("latency", np.min),
    latency_max=("latency", np.max)
)

,latency_mean,latency_std,latency_min,latency_max
observatory,,,,
Fermi,11.609709,14.723566,4.000984,58.956907
ICECUBE,4.098797,0.000997,4.098092,4.099502
INTEGRAL,7.051974,8.621931,4.001710,28.390164
LVK,7.377050,11.013111,4.082593,57.171783
SWIFT,4.227208,0.192336,4.007949,4.367451


### Error area

#### error area global

In [78]:
err_squaredeg = (df["err_arcmin"].values * u.arcmin**2).to_value(u.deg**2)
"{:.3f} ± {:.3f} deg²".format(err_squaredeg.mean(), err_squaredeg.std())

'3146.598 ± 3559.051 deg²'

#### error area / observatory

In [82]:
df["err_squaredeg"] = err_squaredeg
df.groupby("observatory").agg(
    err_area_mean=("err_squaredeg", np.mean),
    err_area_std=("err_squaredeg", np.std),
    err_area_min=("err_squaredeg", np.min),
    err_area_max=("err_squaredeg", np.max)
)

,err_area_mean,err_area_std,err_area_min,err_area_max
observatory,,,,
Fermi,0.311000,2.494939e-01,1.666667e-02,0.833333
ICECUBE,0.083570,1.347981e-02,7.403833e-02,0.093102
INTEGRAL,0.000017,2.402187e-07,1.619444e-05,0.000017
LVK,4233.113734,3.540067e+03,8.146394e+01,24215.300252
SWIFT,0.000002,2.509180e-06,1.388889e-07,0.000005
